In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt

# 加载数据
def load_data(image_file, label_file):
    with open(image_file, 'rb') as f:
        f.read(16)
        images = np.frombuffer(f.read(), np.uint8).reshape(-1, 28, 28)

    with open(label_file, 'rb') as f:
        f.read(8)
        labels = np.frombuffer(f.read(), np.uint8)

    return images, labels

# 计算精确度
def precision(y_true, y_pred):
    class_counts = np.bincount(y_true)
    precisions = []
    for label in np.unique(y_true):
        true_positives = np.sum((y_true == label) & (y_pred == label))
        predicted_positives = np.sum(y_pred == label)
        precision = true_positives / (predicted_positives + 1e-10)
        precisions.append(precision)

    weighted_precision = np.average(precisions, weights=class_counts)
    return weighted_precision

# 计算召回率
def recall(y_true, y_pred):
    class_counts = np.bincount(y_true)
    recalls = []
    for label in np.unique(y_true):
        true_positives = np.sum((y_true == label) & (y_pred == label))
        actual_positives = np.sum(y_true == label)
        recall = true_positives / (actual_positives + 1e-10)
        recalls.append(recall)

    weighted_recall = np.average(recalls, weights=class_counts)
    return weighted_recall

# 计算 F1 分数
def f1_score(y_true, y_pred):
    prec = precision(y_true, y_pred)
    rec = recall(y_true, y_pred)
    f1 = 2 * ((prec * rec) / (prec + rec + 1e-10))
    return f1

# 定义简单的 RNN 模型
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])
        out = self.softmax(out)
        return out

# 加载数据和数据集
train_images_file = 'train-images-idx3-ubyte'
train_labels_file = 'train-labels-idx1-ubyte'
test_images_file = 't10k-images-idx3-ubyte'
test_labels_file = 't10k-labels-idx1-ubyte'

train_images, train_labels = load_data(train_images_file, train_labels_file)
test_images, test_labels = load_data(test_images_file, test_labels_file)

train_images_tensor = torch.Tensor(train_images).type(torch.float32)
train_labels_tensor = torch.LongTensor(train_labels)
test_images_tensor = torch.Tensor(test_images).type(torch.float32)
test_labels_tensor = torch.LongTensor(test_labels)

train_dataset = TensorDataset(train_images_tensor, train_labels_tensor)
test_dataset = TensorDataset(test_images_tensor, test_labels_tensor)

train_loader = DataLoader(train_dataset, batch_size=28, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=28, shuffle=False)

# 初始化模型和优化器
model = SimpleRNN(input_size=28, hidden_size=128, output_size=10)
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 训练模型
def train_model(model, train_loader, optimizer, criterion, epochs=10):
    train_loss_history = []
    train_f1_history = []

    for epoch in range(epochs):
        model.train()
        epoch_losses = []
        for batch_images, batch_labels in train_loader:
            optimizer.zero_grad()
            batch_images = batch_images.view(-1, 28, 28)
            output = model(batch_images)
            loss = criterion(output, batch_labels)
            loss.backward()
            optimizer.step()
            epoch_losses.append(loss.item())

        epoch_loss = np.mean(epoch_losses)
        train_loss_history.append(epoch_loss)

        # 计算 F1 分数并记录历史
        y_true_train, y_pred_train = evaluate_model(model, train_loader)
        train_f1 = f1_score(y_true_train, y_pred_train)
        train_f1_history.append(train_f1)

        print(f'Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss:.4f}, F1 Score: {train_f1:.4f}')

    return train_loss_history, train_f1_history

# 评估模型
def evaluate_model(model, data_loader):
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for images, labels in data_loader:
            images = images.view(-1, 28, 28)
            output = model(images)
            _, predicted = torch.max(output, 1)
            y_true.extend(labels.numpy())
            y_pred.extend(predicted.numpy())
    return y_true, y_pred

# 训练模型并记录历史
train_loss_history, train_f1_history = train_model(model, train_loader, optimizer, criterion, epochs=10)

# 评估模型在训练集和测试集上的性能
y_true_train, y_pred_train = evaluate_model(model, train_loader)
train_f1_score = f1_score(y_true_train, y_pred_train)
print(f'Train F1 Score: {train_f1_score:.4f}')

y_true_test, y_pred_test = evaluate_model(model, test_loader)
test_f1_score = f1_score(y_true_test, y_pred_test)
print(f'Test F1 Score: {test_f1_score:.4f}')

# 绘制训练过程中的损失和 F1 分数曲线
plt.figure(figsize=(10, 6))
plt.plot(train_loss_history, label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss vs. Epochs')
plt.legend()
plt.grid(True)
plt.show()

plt.figure(figsize=(10, 6))
plt.plot(train_f1_history, label='Training F1 Score')
plt.xlabel('Epochs')
plt.ylabel('F1 Score')
plt.title('Training F1 Score vs. Epochs')
plt.legend()
plt.grid(True)
plt.show()
